In [264]:
# import numpy as np
import pandas as pd
# import os
import xgboost as xgb
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import KFold
# from sklearn.metrics import log_loss
# from scipy.interpolate import UnivariateSpline
# import statsmodels.api as sm
# import matplotlib.pyplot as plt
# import collections
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

pd.set_option("display.max_column", 999)

# Load Data

In [265]:
df_mvp = pd.read_excel('../collect-data/src/files/nba_data.xlsx', sheet_name='MVP')
df_mvp.head()

,Unnamed: 0,RANK,FIRST NAME,LAST NAME,SUFFIX,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,SEASON
0,12971,2,Karl,Malone,NaN,UTA,33,82,64,18,2997.7,2249,864,1571,55.0,0,13,0.0,521,690,75.5,193,616,809,368,233,113,48,216,4022,43,1,767,1996-97
1,12531,1,Michael,Jordan,NaN,CHI,35,82,62,20,3182.4,2357,881,1893,46.5,30,126,23.8,565,721,78.4,130,345,475,283,185,141,45,151,3725,5,0,598,1997-98
2,12093,3,Karl,Malone,NaN,UTA,35,49,36,13,1831.6,1164,393,797,49.3,0,1,0.0,378,480,78.8,107,356,463,201,162,62,28,134,2129,21,1,354,1998-99
3,11652,1,Shaquille,O'Neal,NaN,LAL,28,79,66,13,3165.2,2344,956,1665,57.4,0,1,0.0,432,824,52.4,336,742,1078,299,223,36,239,255,4688,63,0,705,1999-00
4,11212,2,Allen,Iverson,NaN,PHI,26,71,50,21,2975.9,2207,762,1813,42.0,98,306,32.0,585,719,81.4,50,223,273,325,237,178,20,147,3379,4,0,371,2000-01


In [266]:
df_all = pd.read_excel('../collect-data/src/files/nba_data.xlsx', sheet_name='NBA Stats')
df_all.head()

,RANK,FIRST NAME,LAST NAME,SUFFIX,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,SEASON
0,453,Nene,NaN,NaN,DEN,23,1,0,1,2.5,0,0,1,0.0,0,0,0.0,0,0,0.0,0,0,0,0,2,0,0,1,-2,0,0,-6,2005-06
1,453,Chris,McCray,NaN,MIL,23,5,1,4,11.8,0,0,3,0.0,0,0,0.0,0,0,0.0,0,0,0,0,2,0,0,1,-2,0,0,-2,2006-07
2,453,Mile,Ilic,NaN,NJN,23,5,3,2,5.9,0,0,3,0.0,0,0,0.0,0,0,0.0,0,1,1,0,3,0,0,1,-2,0,0,-17,2006-07
3,536,Anzejs,Pasecniks,NaN,WAS,25,1,1,0,6.3,0,0,1,0.0,0,1,0.0,0,0,0.0,1,0,1,1,5,0,0,2,-2,0,0,-13,2020-21
4,535,Alondes,Williams,NaN,BKN,24,1,1,0,5.3,0,0,0,0.0,0,0,0.0,0,0,0.0,0,1,1,0,2,0,0,1,-1,0,0,-5,2022-23


# Preprocessing

In [267]:
# Adding 'Is_MVP' column to df_all
df_all['Is_MVP'] = df_all.apply(lambda row: 1 if any(
    (row['LAST NAME'] == mvp['LAST NAME'] and row['SEASON'] == mvp['SEASON']) 
    for _, mvp in df_mvp.iterrows()) else 0, axis=1)

# df_all.drop('Unnamed: 0', axis=1, inplace=True)
df_all.head()

,RANK,FIRST NAME,LAST NAME,SUFFIX,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,SEASON,Is_MVP
0,453,Nene,NaN,NaN,DEN,23,1,0,1,2.5,0,0,1,0.0,0,0,0.0,0,0,0.0,0,0,0,0,2,0,0,1,-2,0,0,-6,2005-06,0
1,453,Chris,McCray,NaN,MIL,23,5,1,4,11.8,0,0,3,0.0,0,0,0.0,0,0,0.0,0,0,0,0,2,0,0,1,-2,0,0,-2,2006-07,0
2,453,Mile,Ilic,NaN,NJN,23,5,3,2,5.9,0,0,3,0.0,0,0,0.0,0,0,0.0,0,1,1,0,3,0,0,1,-2,0,0,-17,2006-07,0
3,536,Anzejs,Pasecniks,NaN,WAS,25,1,1,0,6.3,0,0,1,0.0,0,1,0.0,0,0,0.0,1,0,1,1,5,0,0,2,-2,0,0,-13,2020-21,0
4,535,Alondes,Williams,NaN,BKN,24,1,1,0,5.3,0,0,0,0.0,0,0,0.0,0,0,0.0,0,1,1,0,2,0,0,1,-1,0,0,-5,2022-23,0


## Feature Selection

### Manual

In [390]:
# Current season w/games played and minutes threshold
current_season_stats = df_all[(df_all['SEASON'] == '2023-24') &
                                 (df_all['GP'] >= 55) &
                                 ((df_all['MIN'] / df_all['GP']) > 20)]

# current_season_stats

In [391]:
# Removing those who have less than 1000 PTS a season
more_than_1000 = df_all[(df_all['PTS'] >= 1000) &
                                 (df_all['GP'] >= 55) &
                                 ((df_all['MIN'] / df_all['GP']) > 20)]

# more_than_1000

In [392]:
# Removing those who have greater than 60 rank a season
less_than_60 = df_all[(df_all['RANK'] <= 60) &
                                 (df_all['GP'] >= 55) &
                                 ((df_all['MIN'] / df_all['GP']) > 20)]

# less_than_60

In [393]:
# Removing those who have less than 200 FTA a season
more_than_200 = df_all[(df_all['FTA'] >= 200) &
                                 (df_all['GP'] >= 55) &
                                 ((df_all['MIN'] / df_all['GP']) > 20)]

# more_than_200

In [394]:
# Removing those who have less than 2000 FP a season
more_than_2000 = df_all[(df_all['FP'] >= 2000) &
                                 (df_all['GP'] >= 55) &
                                 ((df_all['MIN'] / df_all['GP']) > 20)]

# more_than_2000

In [395]:
# Removing those who have less than 225 +/- a season
more_than_225 = df_all[(df_all['+/-'].astype(int) >= 225) &
                                 (df_all['GP'] >= 55) &
                                 ((df_all['MIN'] / df_all['GP']) > 20)]
# more_than_225

### XGBoost

In [396]:
# Prepare the data for training
X = df_all.drop(['FIRST NAME', 'LAST NAME', 'SUFFIX', 'TEAM', 'SEASON', 'Is_MVP'], axis=1)
y = df_all['Is_MVP']

In [397]:
# Ensure all columns/features handled correctly
X.rename(columns={'+/-': 'Plus_Minus'}, inplace=True)

X = X.apply(pd.to_numeric, errors='coerce')

In [398]:
# Prep training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [399]:
# Initialize XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

In [400]:
# Train the model
xgb_model.fit(X_train, y_train)

# Get feature importances
feature_importances = xgb_model.feature_importances_
feature_importance_dict = {feature: importance for feature, importance in zip(X.columns, feature_importances)}
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

In [401]:
# sorted_features

# Execution
## Training

In [402]:
# Train according to top features
threshold = 0.01
top_features = [feature for feature, importance in sorted_features if importance > threshold]

X_selected = X[top_features]

# X_selected

In [403]:
# Split the data with the selected features
X_train_selected, X_test_selected, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42, stratify=y)

# Retrain the model with selected features
xgb_model_selected = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model_selected.fit(X_train_selected, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

## Evaluation

In [404]:
# Define hyperparameter grid
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 0.9, 1]
}

# Set up the grid search
grid_search = GridSearchCV(xgb_model_selected, param_grid, cv=3, scoring='accuracy', verbose=1)
grid_search.fit(X_train_selected, y_train)

# Best model
best_model = grid_search.best_estimator_


Fitting 3 folds for each of 81 candidates, totalling 243 fits


In [405]:
# Cross-validate the model
cv_scores = cross_val_score(best_model, X_selected, y, cv=5, scoring='accuracy')
print("CV Mean Score: ", cv_scores.mean())

CV Mean Score:  0.9658600572240186


In [406]:
# Predict on the test set
y_pred = best_model.predict(X_test_selected)

# Evaluation metrics
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy:  0.9977645305514158
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2676
           1       1.00      0.25      0.40         8

    accuracy                           1.00      2684
   macro avg       1.00      0.62      0.70      2684
weighted avg       1.00      1.00      1.00      2684

Confusion Matrix:
 [[2676    0]
 [   6    2]]


# Prediction

In [407]:
current_season_features = pd.get_dummies(current_season_stats)

for col in X_train_selected.columns:
    if col not in current_season_features.columns:
        current_season_features[col] = 0

# Filter to only use top features
current_season_selected = current_season_features[top_features]


In [408]:
# Predict MVP probabilities
predicted_mvp_probabilities = xgb_model_selected.predict_proba(current_season_selected)[:, 1]

current_season_stats['MVP_Probability'] = predicted_mvp_probabilities

top_mvp_candidates = current_season_stats.sort_values(by='MVP_Probability', ascending=False).head(5)

current_season_stats

C:\Users\TreySlice7491\AppData\Local\Temp\ipykernel_40400\3589935121.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_season_stats['MVP_Probability'] = predicted_mvp_probabilities


,RANK,FIRST NAME,LAST NAME,SUFFIX,TEAM,AGE,GP,W,L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-,SEASON,Is_MVP,MVP_Probability
6069,269,Joe,Johnson,NaN,HOU,37,55,35,20,1204.8,372,146,360,40.6,40,145,27.6,40,45,88.9,14,158,172,83,49,18,7,67,729,0,0,-80,2017-18,0,6.568478e-06
6794,239,Justin,Jackson,NaN,SAC,23,68,21,47,1506.3,453,185,419,44.2,57,185,30.8,26,36,72.2,27,160,187,74,40,28,13,81,871,0,0,-238,2017-18,0,7.174041e-07
6895,264,Maurice,Harkless,NaN,POR,25,59,36,23,1263.8,385,147,297,49.5,49,118,41.5,42,59,71.2,46,116,162,53,40,48,43,103,892,0,0,7,2017-18,0,2.150545e-05
6905,243,OG,Anunoby,NaN,TOR,20,74,51,23,1481.5,438,163,346,47.1,73,197,37.1,39,62,62.9,44,140,184,55,45,52,14,130,894,0,0,294,2017-18,0,1.407921e-06
7259,152,Troy,Daniels,NaN,PHX,26,79,20,59,1622.3,703,232,575,40.3,183,458,40.0,56,64,87.5,12,115,127,48,56,26,5,137,964,0,0,-408,2017-18,0,1.535003e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13356,2,James,Harden,NaN,HOU,28,72,59,13,2550.8,2191,651,1449,44.9,265,722,36.7,624,727,85.8,41,348,389,630,315,126,50,169,3816,31,4,522,2017-18,1,4.264216e-01
13361,5,Giannis,Antetokounmpo,NaN,MIL,23,75,39,36,2756.2,2014,742,1402,52.9,43,140,30.7,487,641,76.0,156,597,753,361,223,109,106,231,3881,42,1,148,2017-18,0,1.725995e-04
13393,3,Anthony,Davis,NaN,NOP,25,75,45,30,2727.2,2110,780,1462,53.4,55,162,34.0,495,598,82.8,187,644,831,174,162,115,193,159,4130,50,1,294,2017-18,0,2.412587e-03
13409,4,Russell,Westbrook,NaN,OKC,29,80,47,33,2913.7,2028,757,1687,44.9,97,326,29.8,417,566,73.7,152,652,804,820,381,147,20,200,4343,59,25,384,2017-18,0,3.742450e-03


In [409]:
print('Below are the predictions for MVP with eligibility rules for 2017-18 season:\n')
# print('Below are the predictions for MVP with eligibility rules\nand without those with less than 1000 points:\n')
# print('Below are the predictions for MVP with eligibility rules\nand without those with more than 60 rank:\n')
# print('Below are the predictions for MVP with eligibility rules\nand without those with less than 225 +/-:\n')
print(top_mvp_candidates[['FIRST NAME', 'LAST NAME', 'TEAM', 'MVP_Probability']])

Below are the predictions for MVP with eligibility rules for 2017-18 season:

      FIRST NAME  LAST NAME TEAM  MVP_Probability
13356      James     Harden  HOU         0.426422
13409    Russell  Westbrook  OKC         0.003742
13393    Anthony      Davis  NOP         0.002413
12528      Clint     Capela  HOU         0.001481
13411     LeBron      James  CLE         0.000567
